# Trajectory-Based Continuous Collision Detection (T-CCD) For 2D Particle Systems

The idea here is to develop a mathematical model for the development of T-CCD.

Initial Idea:
- Use the point of intersection to determine if the trajectory of 2 objects has the potential to collide.

Suggested Idea by Sir. Roland:
- Use Newton's second law of motion $(\vec{F} = m\vec{a})$.

Apply this ideas to Continuous Collision Detection (CCD).

## Initial Idea

**Point of Intersection**

- Let $\vec{p_1}$ and $\vec{p_2}$ be the initial positions of the 2 objects.
- Let $\vec{v_1}$ and $\vec{v_2}$ be their velocities.
- Let $t$ be the time when they intersect.

The parametric motion,

$$
\begin{align}
r_1(t) & = \vec{p_1} + \vec{v_1}t \\
r_2(t) & = \vec{p_2} + \vec{v_2}t
\end{align}
$$

The intersection occurs when,

$$
\begin{align}
r_1(t) & = r_2(t) \\
\vec{p_1} + \vec{v_1}t & = \vec{p_2} + \vec{v_2}t
\end{align}
$$

Rearrange the equation to solve for $t$,

$$
(\vec{v_1} - \vec{v_2})t = \vec{p_2} - \vec{p_1}
$$

If the velocities are not parallel ($\vec{v_1} \neq \vec{v_2}$),

$$
t = \frac{\vec{p_2} - \vec{p_1}}{\vec{v_1} - \vec{v_2}}
$$

## Suggested Idea

$$
\vec{F} = m\vec{a}
$$